# Order from Chaos

In [1]:
# from bokeh.plotting import figure, output_file, show
import gzip
import io
import pandas as pd
import urllib.request

%matplotlib inline

import matplotlib # plotting
import matplotlib.pyplot as plt

import sqlite3

%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

## Three Worlds in One

### Normal Python

In [2]:
# unmodified python
x = 'HELLO'
x.lower()

'hello'

In [3]:
# print first two letters
x[0:2]

'HE'

### Pandas

In [4]:
# pandas is very good at handling rows and columns
ramphs = pd.read_csv("http://sebastianheath.com/roman-amphitheaters/roman-amphitheaters.csv")
chronogrps = pd.read_csv("http://sebastianheath.com/roman-amphitheaters/chronogrps.csv")

In [5]:
# print the first two rows of ramphs
ramphs[0:2] # this is just like string indexing

,id,title,label,pleiades,welch,golvin,type,capacity,modcountry,province,...,chronogrp,certainty,youtube,extmajor,extminor,arenamajor,arenaminor,latitude,longitude,elevation
0,duraEuroposAmphitheater,Amphitheater at Dura Europos,Dura,https://pleiades.stoa.org/places/893989,NaN,129.0,amphitheater,1000.0,Syria,syria,...,severan,NaN,NaN,50.0,44.0,31.0,25.0,34.749855,40.728926,223
1,arlesAmphitheater,Amphitheater at Arles,Arles,https://pleiades.stoa.org/places/148217,NaN,154.0,amphitheater,20000.0,France,narbonensis,...,flavian,NaN,https://www.youtube.com/watch?v=oCz-76hb1LU,136.0,107.0,47.0,32.0,43.677778,4.631111,21


In [6]:
# print the first two rows as lower case
ramphs.modcountry.str.lower()[0:2]

0     syria
1    france
Name: modcountry, dtype: object

In [7]:
# but note we haven't changed the column itself
ramphs.modcountry[0:2]

0     Syria
1    France
Name: modcountry, dtype: object

### sqlite

In [8]:
# Now into sqlite
# create the database
ramphs_sql = sqlite3.connect(":memory:")

# ramphs_sql is a python variable that gives access to the sqlite database

In [9]:
# add ramphs and chronogrps tables to the sqlite database
ramphs.to_sql('ramphs',ramphs_sql,if_exists = "replace")
chronogrps.to_sql('chronogrps',ramphs_sql,if_exists = "replace")

In [43]:
# Non pandas way of accessing tables in the database
# create cur as a variable with methods that let us send queries to the sqlite database
cur = ramphs_sql.cursor()

# cur has an .execute method
cur.execute("SELECT id,modcountry FROM ramphs WHERE modcountry = 'France'")
for i in cur.fetchall():
    print(i)

# it is good practice to close a cursor when you're done with it
cur.close()
# the output is not very nicely formatted

('arlesAmphitheater', 'France')
('lyonAmphitheater', 'France')
('parisAmphitheater', 'France')
('nimesAmphitheater', 'France')
('gennesAmphitheater', 'France')
('bordeauxAmphitheater', 'France')
('cimiezAmphitheater', 'France')
('perigueuxAmphitheater', 'France')
('saintesAmphitheater', 'France')
('frejusAmphitheater', 'France')
('toulouseAmphitheater', 'France')
('lillebonnAmphitheater', 'France')
('besanconAmphitheater', 'France')
('limogesAmphitheater', 'France')
('poitiersAmphitheater', 'France')
('reimsAmphitheater', 'France')
('rodezAmphitheater', 'France')
('beziersAmphitheater', 'France')
('agenAmphitheater', 'France')
('narbonneAmphitheater', 'France')
('amiensAmphitheater', 'France')
('meauxAmphitheater', 'France')
('chartresAmphitheater', 'France')
('senlisAmphitheater', 'France')
('toursAmphitheater', 'France')
('autunAmphitheater', 'France')
('stMarcelAmphitheater', 'France')
('metzAmphitheater', 'France')
('grandHalfAmphitheater', 'France')
('bourgesAmphitheater', 'France

In [44]:
# it's simpler with pandas
pd.read_sql_query("SELECT id,modcountry FROM ramphs WHERE modcountry = 'France'", ramphs_sql)
# a single line produces useful result

,id,modcountry
0,arlesAmphitheater,France
1,lyonAmphitheater,France
2,parisAmphitheater,France
3,nimesAmphitheater,France
4,gennesAmphitheater,France
5,bordeauxAmphitheater,France
6,cimiezAmphitheater,France
7,perigueuxAmphitheater,France
8,saintesAmphitheater,France
9,frejusAmphitheater,France


## Joining Tables

In [45]:
# edit this to list the numeric start date of every amphitheater. 
# once you get basic query add an ORDER BY statement to sort by that date.
# note that I've '?'s in the WHERE clause. That's what you need to change.
# we did this in class so I think this will seem familiar
df = pd.read_sql_query('''SELECT ramphs.id, chronogrps.start
    FROM ramphs, chronogrps
    WHERE ramphs.chronogrp = chronogrps.chronogrp
    ORDER by chronogrps.start
    ''',ramphs_sql)
df

,id,start
0,pompeiiAmphitheater,-70
1,cumaeAmphitheater,-70
2,pozzuoliEarlyAmphitheater,-70
3,paestumAmphitheater,-70
4,avellaAmphitheater,-70
5,ferentoAmphitheater,-70
6,sutriumAmphitheater,-70
7,telesiaAmphitheater,-70
8,antiochAmphitheater,-70
9,beneventoAmphitheater,-70


In [61]:
# we ran this query in class
# edit it so it does different things
df = pd.read_sql_query('''SELECT ramphs.id, ramphs.modcountry, chronogrps.start
    FROM ramphs, chronogrps
    WHERE ramphs.chronogrp = chronogrps.chronogrp and ramphs.modcountry = "France"
       AND chronogrps.start > 100
    ORDER BY chronogrps.start
    ''',ramphs_sql)
df

,id,modcountry,start
0,toursAmphitheater,France,117
1,grandHalfAmphitheater,France,200


## Adding Pleiades so we have a lot of data to work with

In [80]:
# get pleiades data
# this transfer takes a while so only execute this cell as necessary
response = urllib.request.urlopen("http://atlantides.org/downloads/pleiades/dumps/pleiades-places-latest.csv.gz")
pleiades = pd.read_csv(io.BytesIO(gzip.decompress(response.read())))


In [81]:
# you can always repeat this cell to reload the pleiades table
pleiades.to_sql('pleiades',ramphs_sql,if_exists = "replace")

In [82]:
# this was where things got whacky in class
cur = ramphs_sql.cursor()
# let's use direct access to the sqlite database to update the pleiades sqlite table
cur.execute("UPDATE pleiades SET path = 'https://pleiades.stoa.org'||path")
cur.close()

# what did we just do? Briefly...
# The '||' combines two string together. The UPDATE command of SQL can change values in all rows of a column

In [83]:
# test that it worked
pd.read_sql_query("SELECT path FROM pleiades LIMIT 5", ramphs_sql)

,path
0,https://pleiades.stoa.org/places/48210385
1,https://pleiades.stoa.org/places/48210386
2,https://pleiades.stoa.org/places/265876
3,https://pleiades.stoa.org/places/265877
4,https://pleiades.stoa.org/places/265878


In [85]:
# find amphitheaters in "Pleiades Roman only" sites
# now ramphs.pleiades shares values with pleiades.path
# we ran this query in class

pd.read_sql_query('''SELECT  ramphs.id, pleiades.timePeriods
          FROM ramphs, pleiades
          WHERE ramphs.pleiades = pleiades.path and pleiades.timePeriods = 'R'
        ORDER BY ramphs.province;''', ramphs_sql)
# this is where we ended in class



,id,timePeriods
0,zagaroloLudus,R
1,metzAmphitheater,R
2,tomen-y-murAmphitheater,R
3,newsteadAmphitheater,R
4,miciaAmphitheater,R
5,porolissumAmphitheater,R
6,sarmizegetusaAmphitheater,R
7,arnsburgAmphitheater,R
8,bobadelaAmphitheater,R
9,tigavaCastraAmphitheater,R


In [87]:
# your intermediate task: edit the following sql so that the chronogrps.start
# column is added to the output.
# That will mean your pulling data from three tables

# HINT: you need to make a change to the SELECT clause, to the FROM clause, and to the WHERE clauses
# but it's the WHERE clause that does the work. Remember, I stressed you can
# add multiple "AND"s. 
pd.read_sql_query('''SELECT  ramphs.id, pleiades.timePeriods, chronogrps.start
          FROM ramphs, pleiades, chronogrps
          WHERE ramphs.pleiades = pleiades.path 
          AND ramphs.chronogrp = chronogrps.chronogrp
          ORDER by chronogrps.start ;''', ramphs_sql)


# MORE HINTS BUT DON"T READ THESE UNTIL YOU'VE TRIED TO DO TASK ON YOUR OWN
# What is the column you need to add to the SELECT clause?
# What is the table you need to add to the FROM clause?
# You need a new " AND ..." 'subclause' of the WHERE clause. One that joins ramphs and chronogrps. We've done this already

,id,timePeriods,start
0,pompeiiAmphitheater,ACHHRM,-70
1,cumaeAmphitheater,ACHRL,-70
2,pozzuoliEarlyAmphitheater,ACHRRLM,-70
3,paestumAmphitheater,ACHRLMM,-70
4,avellaAmphitheater,HRL,-70
5,ferentoAmphitheater,CHRL,-70
6,sutriumAmphitheater,CHRL,-70
7,telesiaAmphitheater,HRL,-70
8,antiochAmphitheater,M1AHHRRRTLLPCEASMRCCIOE1LMM,-70
9,beneventoAmphitheater,HRLMM,-70


In [88]:
# YOUR ASSIGNMENT is either
# 1) Adapt the above SELECT query to do something like:
# "Find all amphitheaters in Italy from sites that Pleiades thinks were founded before 323 BC. Order by earliest date of the amphitheater." 
# Hint: Use the pleiades.minDate column
# Then make some variations on that theme
# OR
# 2) load your own data and write SELECT queries that join tables on common values.

# Add this edited iPyton notebook to your github repository and put a link to it in the Google Doc




In [89]:
pd.read_sql_query('''SELECT  ramphs.modcountry, pleiades.timePeriods, ramphs.id, pleiades.minDate, chronogrps.start
          FROM ramphs, pleiades, chronogrps
          WHERE ramphs.pleiades = pleiades.path AND ramphs.modcountry = "Italy" 
          AND ramphs.chronogrp = chronogrps.chronogrp
          ORDER by chronogrps.start, pleiades.minDate > 323 ;''', ramphs_sql)

,modcountry,timePeriods,id,minDate,start
0,Italy,ACHHRM,pompeiiAmphitheater,-750.0,-70
1,Italy,ACHRL,cumaeAmphitheater,-750.0,-70
2,Italy,ACHRRLM,pozzuoliEarlyAmphitheater,-750.0,-70
3,Italy,ACHRLMM,paestumAmphitheater,-750.0,-70
4,Italy,HRL,avellaAmphitheater,-330.0,-70
5,Italy,CHRL,ferentoAmphitheater,-550.0,-70
6,Italy,CHRL,sutriumAmphitheater,-550.0,-70
7,Italy,HRL,telesiaAmphitheater,-330.0,-70
8,Italy,HRLMM,beneventoAmphitheater,-330.0,-70
9,Italy,CHRL,calviRisortaAmphitheater,-550.0,-70


In [90]:
pd.read_sql_query('''SELECT  ramphs.modcountry, pleiades.timePeriods, ramphs.id, pleiades.minDate, chronogrps.start
          FROM ramphs, pleiades, chronogrps
          WHERE ramphs.pleiades = pleiades.path AND ramphs.modcountry = "Italy"
          AND ramphs.chronogrp = chronogrps.chronogrp
          AND pleiades.minDate < "323"
          ORDER by chronogrps.start ;''', ramphs_sql)

,modcountry,timePeriods,id,minDate,start
0,Italy,ACHHRM,pompeiiAmphitheater,-750.0,-70
1,Italy,ACHRL,cumaeAmphitheater,-750.0,-70
2,Italy,ACHRRLM,pozzuoliEarlyAmphitheater,-750.0,-70
3,Italy,ACHRLMM,paestumAmphitheater,-750.0,-70
4,Italy,HRL,avellaAmphitheater,-330.0,-70
5,Italy,CHRL,ferentoAmphitheater,-550.0,-70
6,Italy,CHRL,sutriumAmphitheater,-550.0,-70
7,Italy,HRL,telesiaAmphitheater,-330.0,-70
8,Italy,HRLMM,beneventoAmphitheater,-330.0,-70
9,Italy,CHRL,calviRisortaAmphitheater,-550.0,-70
